# LLM-Based Pseudo-Factor Analysis (Guenole et al. Methodology)

**Psychometric scale validation using transformer embeddings and exploratory factor analysis**

## Overview

This notebook implements the Pseudo-Factor Analysis (PFA) approach from Guenole et al. for validating psychological scales using pre-trained language model embeddings:

### Key Steps:
1. **Atomic-Reversed Encoding**: Apply item scoring direction (+1/-1) to embeddings
2. **Cosine Similarity Matrix**: Treat normalized similarities as pseudo-correlations
3. **Exploratory Factor Analysis**: ML extraction with oblique rotation
4. **Psychometric Diagnostics**: DAAL, Tucker congruence, KMO/Bartlett tests

### Methodological Choices (Guenole et al.):
- **Extraction**: Maximum Likelihood (ML) - not principal components
- **Rotation**: Oblique (Promax/Oblimin) to allow factor correlations
- **Input**: Cosine similarity of atomic-reversed embeddings
- **Validation**: DAAL, Tucker φ, KMO, Bartlett

### References:
- Guenole, N., et al. (2024). Pre-trained language models for psychometric validation.

---

## Import Dependencies

Import all required libraries for PFA analysis.

In [ ]:
import os
import sys
from datetime import datetime
import warnings

import pandas as pd
import numpy as np
import torch

from sentence_transformers import SentenceTransformer

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from scipy.stats import percentileofscore
import scipy.stats as stats

try:
    from factor_analyzer import FactorAnalyzer
    from factor_analyzer.utils import calculate_kmo, calculate_bartlett_sphericity
    FACTOR_ANALYZER_AVAILABLE = True
except ImportError:
    print("\nWARNING: factor_analyzer not installed!")
    print("Install with: pip install factor-analyzer")
    print("Continuing with limited functionality...\n")
    FACTOR_ANALYZER_AVAILABLE = False

## Configuration

Set all parameters for the analysis.

In [ ]:
# ==============================================================================
# MODEL SELECTION
# ==============================================================================

MODEL_NAMES = [
    "Qwen/Qwen3-Embedding-4B",
]

# ==============================================================================
# PRE-GENERATED EMBEDDINGS
# ==============================================================================

USE_PREGENERATED = False
PREGENERATED_EMBEDDINGS = {
    "4B": "embeddings/dass_items_4B.npz",
}

# ==============================================================================
# SCALE DATA PATH
# ==============================================================================
# Expected CSV columns:
#   - code: item identifier (e.g., "DASS01", "E1")
#   - item: the text of the scale item
#   - factor: theoretical factor label (e.g., "Anxiety", "Extraversion")
#   - scoring: [OPTIONAL] +1 for normal items, -1 for reverse-scored items
#
# If 'scoring' column is missing, all items default to +1 with a warning.
# ==============================================================================

SCALE_CSV_PATH = 'scales/DASS_items.csv'

# ==============================================================================
# EFA SETTINGS
# ==============================================================================

N_FACTORS = None  # None = auto via parallel analysis, or set to integer (e.g., 3)
ROTATION_METHOD = 'promax'  # 'promax', 'oblimin', or 'varimax'
EIGEN_CRITERIA = 'parallel'  # 'parallel' (recommended) or 'eigen1'
PARALLEL_ITER = 100
RANDOM_STATE = 42

# ==============================================================================
# ENSEMBLE ANALYSIS
# ==============================================================================

ENSEMBLE = False  # Set to True to average similarity matrices across models

# ==============================================================================
# EMPIRICAL LOADINGS (optional)
# ==============================================================================

EMPIRICAL_LOADINGS_PATH = None  # e.g., 'empirical_loadings.csv' or None

# ==============================================================================
# OUTPUT
# ==============================================================================

SAVE_DIR = 'results'
os.makedirs(SAVE_DIR, exist_ok=True)

# Set random seeds
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)

# ==============================================================================
# DISPLAY CONFIGURATION
# ==============================================================================

print("Configuration:")
print("="*70)
print(f"Scale: {SCALE_CSV_PATH}")
print(f"Models: {[m.split('/')[-1] for m in MODEL_NAMES]}")
print(f"Factors: {N_FACTORS or 'Auto'}")
print(f"Rotation: {ROTATION_METHOD}")
print(f"Retention: {EIGEN_CRITERIA}")
print(f"Output: {SAVE_DIR}/")
print("="*70)

## Helper Functions

Functions for atomic-reversed encoding, parallel analysis, DAAL, and Tucker congruence.

In [ ]:
def apply_atomic_reversed(embeddings, scoring):
    """Apply atomic-reversed encoding (Guenole et al.)"""
    scoring_array = np.array(scoring).reshape(-1, 1)
    embeddings_signed = embeddings * scoring_array
    norms = np.linalg.norm(embeddings_signed, axis=1)
    zero_norm_items = np.where(norms == 0)[0]
    if len(zero_norm_items) > 0:
        print(f"  WARNING: {len(zero_norm_items)} items have zero norm after signing")
        for idx in zero_norm_items:
            embeddings_signed[idx] = embeddings[idx]
    embeddings_normalized = embeddings_signed / np.linalg.norm(embeddings_signed, axis=1, keepdims=True)
    return embeddings_normalized

def compute_parallel_analysis(corr_matrix, n_iter=100, percentile=95, random_state=42):
    """Parallel analysis for factor retention"""
    np.random.seed(random_state)
    n_items = corr_matrix.shape[0]
    obs_eigenvalues = np.linalg.eigvalsh(corr_matrix)
    obs_eigenvalues = np.sort(obs_eigenvalues)[::-1]
    random_eigenvalues = []
    for _ in range(n_iter):
        random_data = np.random.randn(n_items, n_items)
        random_corr = np.corrcoef(random_data)
        eigs = np.linalg.eigvalsh(random_corr)
        eigs = np.sort(eigs)[::-1]
        random_eigenvalues.append(eigs)
    random_eigenvalues = np.array(random_eigenvalues)
    percentiles = np.percentile(random_eigenvalues, percentile, axis=0)
    n_factors = np.sum(obs_eigenvalues > percentiles)
    return n_factors, obs_eigenvalues, percentiles

def compute_daal(loadings_df, theoretical_factors):
    """Compute DAAL (Dominant Average Absolute Loading)"""
    theoretical_unique = sorted(set(theoretical_factors))
    extracted_factors = loadings_df.columns
    daal_matrix = []
    for ext_factor in extracted_factors:
        row = []
        for theo_factor in theoretical_unique:
            mask = [f == theo_factor for f in theoretical_factors]
            loadings_subset = loadings_df.loc[mask, ext_factor]
            daal_value = loadings_subset.abs().mean()
            row.append(daal_value)
        daal_matrix.append(row)
    daal_df = pd.DataFrame(daal_matrix, index=extracted_factors, columns=theoretical_unique)
    return daal_df

def compute_tucker_congruence(factor_loadings, reference_loadings):
    """Compute Tucker congruence coefficient (phi)"""
    numerator = np.sum(factor_loadings * reference_loadings)
    denom = np.sqrt(np.sum(factor_loadings**2)) * np.sqrt(np.sum(reference_loadings**2))
    return numerator / denom if denom != 0 else 0.0

def create_theoretical_indicators(theoretical_factors, codes):
    """Create indicator matrix for theoretical factors"""
    unique_factors = sorted(set(theoretical_factors))
    indicators = []
    for factor in unique_factors:
        indicator = [1.0 if f == factor else 0.0 for f in theoretical_factors]
        indicators.append(indicator)
    indicators_df = pd.DataFrame(np.array(indicators).T, columns=unique_factors, index=codes)
    return indicators_df

print("✓ Helper functions defined")

## Load and Validate Data

Load scale items from CSV and validate required columns.

In [ ]:
print(f"Loading {SCALE_CSV_PATH}...")
scale = pd.read_csv(SCALE_CSV_PATH)
print(f"Loaded {len(scale)} items")

# Validate columns
required = ['code', 'item', 'factor']
missing = [c for c in required if c not in scale.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

print("✓ Required columns present")

# Handle scoring column
if 'scoring' in scale.columns:
    print("✓ 'scoring' column found")
else:
    print("⚠ WARNING: 'scoring' column missing - defaulting to +1")
    scale['scoring'] = 1

print(f"\nScoring: {(scale['scoring']==1).sum()} normal, {(scale['scoring']==-1).sum()} reverse")
print(f"Factors: {scale['factor'].nunique()} unique")
print(scale['factor'].value_counts().sort_index())

# Extract data
codes = scale['code'].tolist()
items = scale['item'].tolist()
factors = scale['factor'].tolist()
scoring = scale['scoring'].tolist()

print(f"\n✓ Data validated: {len(items)} items, {len(set(factors))} factors")

## Device Detection

In [ ]:
if torch.cuda.is_available():
    device = 'cuda'
    print(f"✓ CUDA: {torch.cuda.get_device_name(0)}")
elif torch.backends.mps.is_available():
    device = 'mps'
    print("✓ Apple MPS")
else:
    device = 'cpu'
    print("Using CPU")

print(f"Device: {device}")

## Load or Generate Embeddings

In [ ]:
all_embeddings = {}
model_sizes = []

for model_name in MODEL_NAMES:
    model_size = model_name.split('-')[-1]
    model_sizes.append(model_size)
    
    print(f"\nModel: {model_name} ({model_size})")
    
    # Try pregenerated
    if USE_PREGENERATED and model_size in PREGENERATED_EMBEDDINGS:
        npz_path = PREGENERATED_EMBEDDINGS[model_size]
        if os.path.exists(npz_path):
            print(f"  Loading from {npz_path}...")
            data = np.load(npz_path, allow_pickle=True)
            embeddings = data['embeddings']
            print(f"  ✓ Loaded: {embeddings.shape}")
            all_embeddings[model_size] = embeddings
            continue
    
    # Generate embeddings
    print(f"  Generating embeddings...")
    model = SentenceTransformer(model_name, device=device)
    embeddings = model.encode(items, show_progress_bar=True, batch_size=8, 
                              convert_to_numpy=True, normalize_embeddings=False)
    print(f"  ✓ Generated: {embeddings.shape}")
    all_embeddings[model_size] = embeddings
    
    # Save
    os.makedirs("embeddings", exist_ok=True)
    save_path = f"embeddings/scale_items_{model_size}.npz"
    np.savez(save_path, embeddings=embeddings, codes=codes, items=items)
    print(f"  ✓ Saved to {save_path}")

print(f"\n✓ All embeddings ready: {list(all_embeddings.keys())}")

## Main PFA Pipeline Function

The core function that runs all 7 steps of the Pseudo-Factor Analysis pipeline.

In [ ]:
def run_pfa_for_model(model_size, embeddings, codes, items, factors, scoring,
                       n_factors=None, rotation='promax', eigen_criteria='parallel',
                       parallel_iter=100, random_state=42, save_dir='results'):
    """
    Run complete Pseudo-Factor Analysis pipeline for one model.

    Args:
        model_size: str, e.g., "4B"
        embeddings: (n_items, dim) array
        codes: list of item codes
        items: list of item texts
        factors: list of theoretical factor labels
        scoring: list of +1/-1 scoring directions
        n_factors: int or None (None = auto via parallel analysis)
        rotation: str, 'promax', 'oblimin', or 'varimax'
        eigen_criteria: 'parallel' or 'eigen1'
        parallel_iter: int, iterations for parallel analysis
        random_state: int, for reproducibility
        save_dir: directory to save results

    Returns:
        results: dict with all results
    """

    print(f"\n{'='*70}")
    print(f"PSEUDO-FACTOR ANALYSIS: {model_size}")
    print('='*70)

    results = {'model_size': model_size}

    # Step 1: Atomic-reversed encoding
    print("\n[1/7] Applying atomic-reversed encoding...")
    embeddings_ar = apply_atomic_reversed(embeddings, scoring)
    print(f"  ✓ Shape: {embeddings_ar.shape}")

    np.save(f"{save_dir}/embeddings_atomic_reversed_{model_size}.npy", embeddings_ar)

    # Step 2: Cosine similarity matrix
    print("\n[2/7] Computing cosine similarity matrix...")
    sim_matrix = cosine_similarity(embeddings_ar)
    print(f"  ✓ Shape: {sim_matrix.shape}")

    np.save(f"{save_dir}/similarity_{model_size}.npy", sim_matrix)
    sim_df = pd.DataFrame(sim_matrix, index=codes, columns=codes)
    sim_df.to_csv(f"{save_dir}/similarity_{model_size}.csv")

    results['similarity_matrix'] = sim_matrix

    # Step 3: Sampling adequacy tests
    print("\n[3/7] Computing KMO and Bartlett test...")

    if FACTOR_ANALYZER_AVAILABLE:
        kmo_per_item, kmo_total = calculate_kmo(sim_matrix)
        print(f"  ✓ KMO overall: {kmo_total:.3f}")

        if kmo_total < 0.5:
            print("    ⚠ WARNING: KMO < 0.5 (unacceptable)")
        elif kmo_total < 0.6:
            print("    ⚠ WARNING: KMO < 0.6 (poor)")
        elif kmo_total < 0.7:
            print("    KMO is mediocre")
        elif kmo_total < 0.8:
            print("    KMO is middling")
        elif kmo_total < 0.9:
            print("    KMO is meritorious")
        else:
            print("    KMO is marvelous")

        chi_square, p_value = calculate_bartlett_sphericity(sim_matrix)
        print(f"  ✓ Bartlett: χ²={chi_square:.2f}, p={p_value:.4e}")

        if p_value > 0.05:
            print("    ⚠ WARNING: Not significant (p > 0.05)")
        else:
            print("    ✓ Significant (p < 0.05)")

        results['kmo_total'] = kmo_total
        results['kmo_per_item'] = kmo_per_item
        results['bartlett_chi2'] = chi_square
        results['bartlett_p'] = p_value
    else:
        print("  ⚠ Skipping (factor_analyzer not available)")

    # Step 4: Parallel analysis for factor retention
    print("\n[4/7] Determining number of factors...")

    if n_factors is None:
        if eigen_criteria == 'parallel':
            print(f"  Running parallel analysis ({parallel_iter} iterations)...")
            n_factors_auto, obs_eigs, percentile_eigs = compute_parallel_analysis(
                sim_matrix, n_iter=parallel_iter, random_state=random_state
            )
            print(f"  ✓ Suggested {n_factors_auto} factors")
            n_factors = max(1, n_factors_auto)
            results['observed_eigenvalues'] = obs_eigs
            results['percentile_eigenvalues'] = percentile_eigs
        else:  # eigen1
            eigs = np.linalg.eigvalsh(sim_matrix)
            eigs = np.sort(eigs)[::-1]
            n_factors = np.sum(eigs > 1)
            print(f"  ✓ Kaiser rule (eigen>1): {n_factors} factors")
            results['observed_eigenvalues'] = eigs

    print(f"  ✓ Extracting {n_factors} factors with {rotation} rotation")
    results['n_factors'] = n_factors

    # Step 5: Exploratory Factor Analysis
    print("\n[5/7] Running Exploratory Factor Analysis...")

    if not FACTOR_ANALYZER_AVAILABLE:
        print("  ⚠ ERROR: factor_analyzer not available!")
        print("  Install: pip install factor-analyzer")
        return results

    fa = FactorAnalyzer(
        n_factors=n_factors,
        rotation=rotation,
        method='ml',
        rotation_kwargs={'normalize': True} if rotation in ['promax', 'oblimin'] else {}
    )

    fa.fit(sim_matrix)
    print("  ✓ EFA complete")

    loadings = fa.loadings_
    communalities = fa.get_communalities()
    uniquenesses = fa.get_uniquenesses()
    variance = fa.get_factor_variance()

    factor_names = [f"Factor{i+1}" for i in range(n_factors)]
    loadings_df = pd.DataFrame(loadings, index=codes, columns=factor_names)

    print(f"  Loadings shape: {loadings.shape}")
    print(f"  Variance explained (cumulative): {variance[2][-1]:.1%}")

    loadings_df.to_csv(f"{save_dir}/loadings_{model_size}.csv")

    variance_df = pd.DataFrame(variance, index=['SS Loadings', 'Proportion', 'Cumulative'])
    variance_df.to_csv(f"{save_dir}/variance_{model_size}.csv")

    communalities_df = pd.DataFrame({
        'communality': communalities,
        'uniqueness': uniquenesses
    }, index=codes)
    communalities_df.to_csv(f"{save_dir}/communalities_{model_size}.csv")

    results['loadings'] = loadings_df
    results['variance'] = variance
    results['communalities'] = communalities
    results['uniquenesses'] = uniquenesses

    # Step 6: DAAL diagnostic
    print("\n[6/7] Computing DAAL...")

    daal_df = compute_daal(loadings_df, factors)
    print(f"  ✓ DAAL matrix: {daal_df.shape}")

    assignments = []
    for ext_factor in daal_df.index:
        best_theo = daal_df.loc[ext_factor].idxmax()
        best_daal = daal_df.loc[ext_factor, best_theo]
        assignments.append({
            'extracted': ext_factor,
            'assigned_to': best_theo,
            'daal': best_daal
        })

    assignments_df = pd.DataFrame(assignments)
    print("\n  Factor assignments (DAAL):")
    for _, row in assignments_df.iterrows():
        print(f"    {row['extracted']} → {row['assigned_to']} (DAAL={row['daal']:.3f})")

    daal_df.to_csv(f"{save_dir}/daal_{model_size}.csv")

    results['daal'] = daal_df
    results['daal_assignments'] = assignments_df

    # Step 7: Tucker congruence
    print("\n[7/7] Computing Tucker congruence...")

    theoretical_indicators = create_theoretical_indicators(factors, codes)

    tucker_matrix = []
    for ext_factor in factor_names:
        row = []
        for theo_factor in theoretical_indicators.columns:
            phi = compute_tucker_congruence(
                loadings_df[ext_factor].values,
                theoretical_indicators[theo_factor].values
            )
            row.append(phi)
        tucker_matrix.append(row)

    tucker_df = pd.DataFrame(
        tucker_matrix,
        index=factor_names,
        columns=theoretical_indicators.columns
    )

    print(f"  ✓ Tucker matrix: {tucker_df.shape}")
    print("\n  Interpretation guide:")
    print("    φ ≥ .95: Excellent agreement")
    print("    φ ≥ .85: Fair agreement")
    print("    φ < .85: Poor agreement")

    tucker_best = []
    for ext_factor in tucker_df.index:
        best_theo = tucker_df.loc[ext_factor].idxmax()
        best_phi = tucker_df.loc[ext_factor, best_theo]
        tucker_best.append({
            'extracted': ext_factor,
            'best_match': best_theo,
            'tucker_phi': best_phi
        })

    tucker_best_df = pd.DataFrame(tucker_best)
    print("\n  Best matches (Tucker φ):")
    for _, row in tucker_best_df.iterrows():
        print(f"    {row['extracted']} ↔ {row['best_match']} (φ={row['tucker_phi']:.3f})")

    tucker_df.to_csv(f"{save_dir}/tucker_{model_size}.csv")

    results['tucker'] = tucker_df
    results['tucker_best'] = tucker_best_df

    # Combined diagnostics
    diagnostics = {
        'model': model_size,
        'n_items': len(codes),
        'n_factors_extracted': n_factors,
        'rotation': rotation,
    }

    if FACTOR_ANALYZER_AVAILABLE:
        diagnostics['kmo'] = kmo_total
        diagnostics['bartlett_p'] = p_value

    diagnostics['variance_explained'] = variance[2][-1]

    diagnostics_df = pd.DataFrame([diagnostics])
    diagnostics_df.to_csv(f"{save_dir}/diagnostics_{model_size}.csv", index=False)

    results['diagnostics'] = diagnostics_df

    print(f"\n{'='*70}")
    print(f"✓ PFA COMPLETE FOR {model_size}")
    print('='*70)

    return results

## Visualization Function

Create comprehensive visualizations of PFA results.

In [ ]:
def create_visualizations(results, factors, codes, model_size, save_dir='results'):
    """Create all visualizations for PFA results"""

    print(f"\nCreating visualizations for {model_size}...")

    fig = plt.figure(figsize=(20, 12))

    # 1. Scree plot
    ax1 = plt.subplot(2, 3, 1)
    if 'observed_eigenvalues' in results:
        obs_eigs = results['observed_eigenvalues']
        n_show = min(20, len(obs_eigs))
        ax1.plot(range(1, n_show + 1), obs_eigs[:n_show], 'o-', label='Observed', linewidth=2)

        if 'percentile_eigenvalues' in results:
            perc_eigs = results['percentile_eigenvalues']
            ax1.plot(range(1, n_show + 1), perc_eigs[:n_show], 's--',
                    label='95th percentile (random)', linewidth=2)

        ax1.axhline(1, color='red', linestyle='--', alpha=0.5, label='Eigen = 1')
        ax1.set_xlabel('Factor Number')
        ax1.set_ylabel('Eigenvalue')
        ax1.set_title(f'Scree Plot - {model_size}')
        ax1.legend()
        ax1.grid(True, alpha=0.3)

    # 2. Similarity matrix heatmap
    ax2 = plt.subplot(2, 3, 2)
    if 'similarity_matrix' in results:
        sim = results['similarity_matrix']
        factor_order = sorted(range(len(factors)), key=lambda i: (factors[i], i))
        sim_ordered = sim[factor_order][:, factor_order]

        sns.heatmap(sim_ordered, cmap='RdBu_r', center=0, vmin=-1, vmax=1,
                   square=True, ax=ax2, cbar_kws={'label': 'Cosine Similarity'})
        ax2.set_title(f'Similarity Matrix - {model_size}')
        ax2.set_xlabel('Items')
        ax2.set_ylabel('Items')

    # 3. Loadings heatmap
    ax3 = plt.subplot(2, 3, 3)
    if 'loadings' in results:
        loadings_df = results['loadings']
        factor_order = sorted(range(len(factors)), key=lambda i: (factors[i], i))
        ordered_loadings = loadings_df.loc[[codes[i] for i in factor_order]]

        sns.heatmap(ordered_loadings.values, cmap='RdBu_r', center=0,
                   vmin=-1, vmax=1, ax=ax3, cbar_kws={'label': 'Loading'})
        ax3.set_title(f'Factor Loadings - {model_size}')
        ax3.set_xlabel('Extracted Factors')
        ax3.set_ylabel('Items')
        ax3.set_yticks([])

    # 4. DAAL heatmap
    ax4 = plt.subplot(2, 3, 4)
    if 'daal' in results:
        daal = results['daal']
        sns.heatmap(daal.values, annot=True, fmt='.3f', cmap='YlOrRd',
                   xticklabels=daal.columns, yticklabels=daal.index,
                   ax=ax4, cbar_kws={'label': 'DAAL'})
        ax4.set_title(f'DAAL Matrix - {model_size}')
        ax4.set_xlabel('Theoretical Factors')
        ax4.set_ylabel('Extracted Factors')

    # 5. Tucker congruence heatmap
    ax5 = plt.subplot(2, 3, 5)
    if 'tucker' in results:
        tucker = results['tucker']
        sns.heatmap(tucker.values, annot=True, fmt='.3f', cmap='YlGnBu',
                   xticklabels=tucker.columns, yticklabels=tucker.index,
                   ax=ax5, vmin=0, vmax=1, cbar_kws={'label': 'Tucker φ'})
        ax5.set_title(f'Tucker Congruence - {model_size}')
        ax5.set_xlabel('Theoretical Factors')
        ax5.set_ylabel('Extracted Factors')

    # 6. Tucker best matches bar plot
    ax6 = plt.subplot(2, 3, 6)
    if 'tucker_best' in results:
        tucker_best = results['tucker_best']
        colors = ['green' if x >= 0.95 else 'orange' if x >= 0.85 else 'red'
                 for x in tucker_best['tucker_phi']]
        ax6.barh(tucker_best['extracted'], tucker_best['tucker_phi'], color=colors)
        ax6.axvline(0.95, color='green', linestyle='--', alpha=0.5, label='Excellent (≥.95)')
        ax6.axvline(0.85, color='orange', linestyle='--', alpha=0.5, label='Fair (≥.85)')
        ax6.set_xlabel('Tucker φ')
        ax6.set_ylabel('Extracted Factor')
        ax6.set_title(f'Best Tucker Congruence - {model_size}')
        ax6.legend()
        ax6.set_xlim(0, 1)

    plt.tight_layout()
    plt.savefig(f'{save_dir}/visualizations_{model_size}.png', dpi=150, bbox_inches='tight')
    plt.show()

    print(f"  ✓ Saved: {save_dir}/visualizations_{model_size}.png")

## Run PFA for All Models

Execute the complete pipeline for each model.

In [ ]:
all_results = {}

for model_size in model_sizes:
    embeddings = all_embeddings[model_size]
    
    results = run_pfa_for_model(
        model_size=model_size,
        embeddings=embeddings,
        codes=codes,
        items=items,
        factors=factors,
        scoring=scoring,
        n_factors=N_FACTORS,
        rotation=ROTATION_METHOD,
        eigen_criteria=EIGEN_CRITERIA,
        parallel_iter=PARALLEL_ITER,
        random_state=RANDOM_STATE,
        save_dir=SAVE_DIR
    )
    
    all_results[model_size] = results
    
    # Create visualizations
    create_visualizations(results, factors, codes, model_size, save_dir=SAVE_DIR)

print(f"\n{'='*70}")
print("✓ PFA COMPLETE FOR ALL MODELS")
print('='*70)

## Analyze Nearest Neighbors - All Models

Examine nearest neighbors in the original high-dimensional embedding space to validate semantic clustering.

In [ ]:
# Select a sample item to analyze
sample_idx = 0  # First item

print("Finding nearest neighbors in original embedding space...")
print("=" * 70)
print(f"\nSample item #{sample_idx}:")
print(f"  Code: {codes[sample_idx]}")
print(f"  Factor: {factors[sample_idx]}")
print(f"  Text: {items[sample_idx]}")

for model_size in model_sizes:
    embeddings = all_embeddings[model_size]
    print(f"\n{'='*70}")
    print(f"{model_size} Model - Original {embeddings.shape[1]}D Space")
    print(f"{'='*70}")
    
    # Compute cosine similarity between sample and all items
    # Note: embeddings are already processed through atomic-reversed encoding
    # For pure nearest neighbors, we could use original embeddings, but using
    # the processed ones ensures consistency with the PFA analysis
    similarities = cosine_similarity([embeddings[sample_idx]], embeddings)[0]
    
    # Find 5 most similar items (excluding itself)
    most_similar_indices = np.argsort(similarities)[::-1][1:6]
    
    print(f"5 Most similar items (by cosine similarity):")
    for rank, idx in enumerate(most_similar_indices, 1):
        print(f"  {rank}. [{factors[idx]}] {items[idx][:80]}...")
        print(f"      Similarity: {similarities[idx]:.4f}")

print(f"\n{'='*70}")
print("✓ Nearest neighbors analysis complete")
print('='*70)

## Within- vs Between-Construct Similarity Analysis (Milano et al. 2025)

Test whether items from the same theoretical factor are more similar than items from different factors.

**Method:**
- **Within-construct similarities**: Cosine similarities between items sharing the same factor
- **Between-construct similarities**: Cosine similarities between items from different factors
- **Statistical test**: Welch's t-test (unequal variances)
- **Effect size**: Cohen's d

This analysis provides empirical evidence for **construct validity**: if embeddings capture the theoretical factor structure, within-construct similarities should be significantly higher than between-construct similarities.

In [ ]:
print("\n" + "="*70)
print("MILANO ET AL. (2025) WITHIN- vs BETWEEN-CONSTRUCT ANALYSIS")
print("="*70)

from scipy.stats import ttest_ind

for model_size in model_sizes:
    print(f"\n{model_size} Model:")
    print("-" * 70)
    
    # Get similarity matrix from results
    sim_matrix = all_results[model_size]['similarity_matrix']
    n_items = len(factors)
    
    # Collect similarities (upper triangle only, i < j)
    within_sims = []
    between_sims = []
    
    for i in range(n_items):
        for j in range(i + 1, n_items):  # Upper triangle only
            sim = sim_matrix[i, j]
            
            if factors[i] == factors[j]:
                within_sims.append(sim)
            else:
                between_sims.append(sim)
    
    # Convert to arrays
    within_sims = np.array(within_sims)
    between_sims = np.array(between_sims)
    
    # Compute statistics
    mean_within = np.mean(within_sims)
    sd_within = np.std(within_sims, ddof=1)
    mean_between = np.mean(between_sims)
    sd_between = np.std(between_sims, ddof=1)
    mean_diff = mean_within - mean_between
    
    # Cohen's d (pooled standard deviation)
    pooled_sd = np.sqrt((sd_within**2 + sd_between**2) / 2)
    cohens_d = mean_diff / pooled_sd if pooled_sd > 0 else 0.0
    
    # Welch's t-test (unequal variances)
    t_stat, p_value = ttest_ind(within_sims, between_sims, equal_var=False)
    
    # Degrees of freedom (Welch-Satterthwaite)
    n1, n2 = len(within_sims), len(between_sims)
    s1_sq, s2_sq = sd_within**2, sd_between**2
    df = (s1_sq/n1 + s2_sq/n2)**2 / ((s1_sq/n1)**2/(n1-1) + (s2_sq/n2)**2/(n2-1))
    
    # Print results
    print(f"  Within-construct:  M = {mean_within:.3f}, SD = {sd_within:.3f}, n = {len(within_sims)}")
    print(f"  Between-construct: M = {mean_between:.3f}, SD = {sd_between:.3f}, n = {len(between_sims)}")
    print(f"  Mean difference: {mean_diff:.3f}")
    print(f"  Welch's t({df:.1f}) = {t_stat:.2f}, p = {p_value:.4e}")
    print(f"  Cohen's d = {cohens_d:.3f}")
    
    # Interpretation
    if p_value < 0.001:
        sig_text = "highly significant (p < .001)"
    elif p_value < 0.01:
        sig_text = "very significant (p < .01)"
    elif p_value < 0.05:
        sig_text = "significant (p < .05)"
    else:
        sig_text = "not significant (p ≥ .05)"
    
    if abs(cohens_d) >= 0.8:
        effect_text = "large effect"
    elif abs(cohens_d) >= 0.5:
        effect_text = "medium effect"
    elif abs(cohens_d) >= 0.2:
        effect_text = "small effect"
    else:
        effect_text = "negligible effect"
    
    print(f"\n  Interpretation: {sig_text}, {effect_text}")
    
    if mean_within > mean_between and p_value < 0.05:
        print(f"  ✓ Items from the same factor are significantly more similar.")
    elif mean_within < mean_between and p_value < 0.05:
        print(f"  ⚠ Items from different factors are MORE similar (unexpected!).")
    else:
        print(f"  ⚠ No significant difference in within- vs between-construct similarity.")

print("\n" + "="*70)
print("✓ MILANO ET AL. ANALYSIS COMPLETE")
print("="*70)

In [ ]:
# Visualize within vs between similarities
print("\nCreating Milano et al. visualization...")

for model_size in model_sizes:
    sim_matrix = all_results[model_size]['similarity_matrix']
    n_items = len(factors)
    
    # Collect similarities
    within_sims = []
    between_sims = []
    
    for i in range(n_items):
        for j in range(i + 1, n_items):
            sim = sim_matrix[i, j]
            if factors[i] == factors[j]:
                within_sims.append(sim)
            else:
                between_sims.append(sim)
    
    # Create violin plot
    fig, ax = plt.subplots(figsize=(10, 6))
    
    data = [within_sims, between_sims]
    positions = [1, 2]
    
    parts = ax.violinplot(data, positions=positions, showmeans=True, showmedians=True)
    
    ax.set_xticks(positions)
    ax.set_xticklabels(['Within-Construct', 'Between-Construct'])
    ax.set_ylabel('Cosine Similarity')
    ax.set_title(f'Milano et al. (2025) Analysis - {model_size}\nWithin- vs Between-Construct Similarities')
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add mean values as text
    mean_within = np.mean(within_sims)
    mean_between = np.mean(between_sims)
    
    ax.text(1, mean_within, f'M={mean_within:.3f}', 
            ha='center', va='bottom', fontsize=10, fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    ax.text(2, mean_between, f'M={mean_between:.3f}', 
            ha='center', va='bottom', fontsize=10, fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Add sample sizes
    ax.text(1, ax.get_ylim()[0], f'n={len(within_sims)}', 
            ha='center', va='top', fontsize=9, style='italic')
    ax.text(2, ax.get_ylim()[0], f'n={len(between_sims)}', 
            ha='center', va='top', fontsize=9, style='italic')
    
    plt.tight_layout()
    plt.savefig(f'{SAVE_DIR}/milano_analysis_{model_size}.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"  ✓ Saved: {SAVE_DIR}/milano_analysis_{model_size}.png")

print("\n✓ Milano et al. visualizations complete")

## Summary Report

Display key diagnostics and results for all models.

In [ ]:
print("\n" + "="*70)
print("SUMMARY")
print("="*70)

for model_size in model_sizes:
    results = all_results[model_size]
    print(f"\n{model_size}:")
    print("-" * 70)
    
    if 'diagnostics' in results:
        diag = results['diagnostics'].iloc[0]
        print(f"  Items: {diag['n_items']}")
        print(f"  Factors: {diag['n_factors_extracted']}")
        print(f"  Rotation: {diag['rotation']}")
        if 'kmo' in diag:
            print(f"  KMO: {diag['kmo']:.3f}")
        if 'bartlett_p' in diag:
            print(f"  Bartlett p: {diag['bartlett_p']:.4e}")
        print(f"  Variance: {diag['variance_explained']:.1%}")
    
    if 'daal_assignments' in results:
        print("\n  DAAL assignments:")
        for _, row in results['daal_assignments'].iterrows():
            print(f"    {row['extracted']} → {row['assigned_to']} (DAAL={row['daal']:.3f})")
    
    if 'tucker_best' in results:
        print("\n  Tucker congruence:")
        for _, row in results['tucker_best'].iterrows():
            print(f"    {row['extracted']} ↔ {row['best_match']} (φ={row['tucker_phi']:.3f})")

print("\n" + "="*70)
print("✓ ANALYSIS COMPLETE")
print("="*70)
print(f"\nResults saved to: {SAVE_DIR}/")

## Automatic Factor Naming with LLM

Generate semantic names for EFA-extracted factors using Qwen2.5-1.5B-Instruct.

**Approach:**
1. For each extracted factor, identify the top 10 items with highest absolute loadings
2. Extract the text of these high-loading items
3. Feed the item text to Qwen2.5-1.5B-Instruct LLM with a prompt
4. LLM generates a concise semantic label (single word or short phrase) describing what the items measure
5. Store factor name mappings in `factor_name_mappings` dictionary

**Prompt Format:**
```
"These are items from a psychological scale that all load strongly on the same latent factor: [item1 | item2 | ...]. 
Provide a single word or very short phrase that best describes the psychological construct these items are measuring."
```

**Advantages over vocabulary-based naming:**
- Uses actual scale items that define each factor (not external vocabulary)
- More psychologically valid (factors are defined by their indicators)
- No need to load separate word embeddings
- Direct relationship between loadings and names

**Note:** Generated names are stored in the `factor_name_mappings` dictionary. Original results in `all_results` remain unchanged.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import re

print("Loading Qwen/Qwen2.5-1.5B-Instruct...")
llm_model_name = "Qwen/Qwen2.5-1.5B-Instruct"

try:
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
    
    # Load model and move to appropriate device
    llm_model = AutoModelForCausalLM.from_pretrained(
        llm_model_name,
        dtype=torch.float32,
        low_cpu_mem_usage=True
    )
    
    llm_model = llm_model.to(device)
    llm_model.eval()
    
    print(f"✓ Qwen/Qwen2.5-1.5B-Instruct loaded successfully!")
    print(f"  Device: {device}")
    
except Exception as e:
    print(f"\n✗ Error loading {llm_model_name}:")
    print(f"  {type(e).__name__}: {str(e)}")
    print(f"\nSkipping automatic factor naming...")
    llm_model = None

# Proceed with factor naming if model loaded successfully
if llm_model is not None:
    print(f"\nGenerating semantic names for extracted factors...\n")
    
    # Define system prompt
    system_prompt = "You are a helpful assistant that provides concise, one or two-word summaries."
    print(f"System prompt: {system_prompt}\n")
    
    # Store factor name mappings
    factor_name_mappings = {}
    
    # Process each model
    for model_size in model_sizes:
        print(f"{'='*70}")
        print(f"{model_size} Model - Automatic Factor Naming")
        print(f"{'='*70}\n")
        
        # Get loadings for this model
        loadings_df = all_results[model_size]['loadings']
        n_factors = loadings_df.shape[1]
        
        factor_name_mappings[model_size] = {}
        
        # Process each factor
        for factor_name in loadings_df.columns:
            print(f"\n{factor_name}:")
            print("-"*70)
            
            # Get absolute loadings for this factor (to find strongest indicators)
            factor_loadings = loadings_df[factor_name].abs()
            
            # Find top 10 items with highest absolute loadings
            top_indices = factor_loadings.nlargest(10).index
            
            # Get the actual item text for these top-loading items
            top_items_text = []
            print(f"Top loading items:")
            for i, code in enumerate(top_indices, 1):
                item_idx = codes.index(code)
                item_text = items[item_idx]
                loading_val = loadings_df.loc[code, factor_name]
                top_items_text.append(item_text)
                print(f"  {i}. [{code}] (λ={loading_val:.3f}): {item_text[:70]}...")
            
            # Create prompt from top 5 items (truncate for context length)
            items_for_prompt = " | ".join([item[:120] for item in top_items_text[:5]])
            
            user_prompt = f"""These are items from a psychological scale that all load strongly on the same latent factor:

{items_for_prompt}

Provide a single word or very short phrase (maximum 3 words) that best describes the psychological construct these items are measuring. Provide ONLY the label, nothing else."""
            
            # Create messages for chat template
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
            
            # Apply chat template
            text = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
            
            # Tokenize
            inputs = tokenizer([text], return_tensors="pt")
            inputs = {k: v.to(device) for k, v in inputs.items()}
            
            # Generate with greedy decoding
            with torch.no_grad():
                outputs = llm_model.generate(
                    **inputs,
                    max_new_tokens=10,
                    do_sample=False,
                    pad_token_id=tokenizer.eos_token_id
                )
            
            # Decode only the new tokens
            generated_text = tokenizer.decode(
                outputs[0][inputs['input_ids'].shape[1]:], 
                skip_special_tokens=True
            )
            
            # Extract and clean the generated name
            generated_name = generated_text.strip()
            generated_name = re.sub(r'^["\'\s]+|["\'\s]+$', '', generated_name)
            generated_name = re.sub(r'[.,;:!?]+$', '', generated_name)
            generated_name = generated_name.split('\n')[0].strip()
            
            if len(generated_name) > 50:
                generated_name = generated_name[:50].strip()
            
            # If generation failed or is empty, keep original name
            if not generated_name or len(generated_name) < 2:
                generated_name = factor_name
            
            print(f"\nGenerated name: '{generated_name}'")
            
            # Store the mapping
            factor_name_mappings[model_size][factor_name] = generated_name
    
    print("\n" + "="*70)
    print("✓ Automatic factor naming complete!")
    print("="*70)
    
    # Display summary of all mappings
    print("\nFactor Name Mappings Summary:")
    print("="*70)
    for model_size in model_sizes:
        print(f"\n{model_size}:")
        for old_name, new_name in factor_name_mappings[model_size].items():
            print(f"  {old_name} → '{new_name}'")
    
    # Optionally update the results with new names
    print("\nNote: Factor names are stored in 'factor_name_mappings' dictionary.")
    print("Original results in 'all_results' remain unchanged.")
    
else:
    print("\n⚠ Skipping automatic factor naming due to model loading failure.")

## T-SNE Visualizations: Original Factors vs EFA-Extracted Factors

Create T-SNE visualizations comparing original theoretical factor assignments with EFA-extracted factor assignments.

**Two-plot comparison:**
- **Plot 1 (Top)**: Items colored by original theoretical factors (from CSV 'factor' column)
- **Plot 2 (Bottom)**: Items colored by extracted EFA factors (assigned by highest loading)
  - Uses LLM-generated factor names if available
  - Falls back to "Factor1", "Factor2", etc. if LLM naming hasn't run

**Visualization details:**
- 2D visualization via t-SNE dimensionality reduction
- Color-coded by factor with item codes labeled
- Separate files saved for each visualization type

In [ ]:
# Create factor assignments based on highest loadings
print("Creating factor assignments from EFA loadings...")
print("="*70)

factor_assignments = {}

for model_size in model_sizes:
    loadings_df = all_results[model_size]['loadings']
    
    # For each item, find the factor with highest absolute loading
    item_to_factor = {}
    
    for item_code in loadings_df.index:
        # Get absolute loadings for this item across all factors
        abs_loadings = loadings_df.loc[item_code].abs()
        # Find factor with highest loading
        assigned_factor = abs_loadings.idxmax()
        item_to_factor[item_code] = assigned_factor
    
    # Group items by their assigned factors
    factor_items = {}
    for factor_name in loadings_df.columns:
        # Find all items assigned to this factor
        assigned_items = [
            {'index': codes.index(code), 'code': code}
            for code, assigned_f in item_to_factor.items()
            if assigned_f == factor_name
        ]
        factor_items[factor_name] = assigned_items
    
    factor_assignments[model_size] = factor_items
    
    # Display summary
    print(f"\n{model_size}:")
    for factor_name, items_list in factor_items.items():
        # Get display name (LLM-generated or default)
        if 'factor_name_mappings' in dir() and model_size in factor_name_mappings:
            display_name = factor_name_mappings[model_size].get(factor_name, factor_name)
        else:
            display_name = factor_name
        
        print(f"  {factor_name} ('{display_name}'): {len(items_list)} items")

print("\n" + "="*70)
print("✓ Factor assignments created!")
print("="*70)

In [ ]:
# Import visualization libraries
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# Prepare data for T-SNE (same across all models)
print("Preparing data for T-SNE...")
print(f"Number of items: {len(factors)}")

# Get unique factors for legend
print(f"Factors: {unique_factors}")

# Create a color map for the personality factors
import matplotlib.cm as cm
colors_map = cm.get_cmap('tab10', len(unique_factors))
factor_to_color = {factor: colors_map(i) for i, factor in enumerate(unique_factors)}

# Run T-SNE and create visualizations for all models
print("Running T-SNE for all models...")
print("=" * 70)

all_tsne_embeddings = {}

for model_size, embeddings in all_embeddings.items():
    print(f"\n{'='*70}")
    print(f"Running T-SNE for {model_size} model...")
    print(f"{'='*70}")
    print(f"Input shape: {embeddings.shape}")
    
    # Run T-SNE dimensionality reduction
    tsne = TSNE(
        n_components=2,      # Reduce to 2D
        perplexity=25,       # Balance local vs global structure
        max_iter=1000,       # Number of iterations
        random_state=42,     # For reproducibility
        verbose=1            # Show progress
    )
    
    # Transform high-D embeddings to 2D
    embeddings_2d = tsne.fit_transform(embeddings)
    all_tsne_embeddings[model_size] = embeddings_2d
    
    print(f"✓ T-SNE complete! 2D embeddings shape: {embeddings_2d.shape}")

print(f"\n{'='*70}")
print(f"✓ T-SNE complete for all {len(all_tsne_embeddings)} models!")
print(f"{'='*70}")

# Create T-SNE scatter plots for all models
print("Creating T-SNE visualizations...")
print("=" * 70)

# Create plots directory if it doesn't exist
plots_dir = "plots"
os.makedirs(plots_dir, exist_ok=True)
print(f"Plots will be saved to: {plots_dir}/")

# Generate timestamp for filename
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Determine number of models
num_models = len(all_tsne_embeddings)
print(f"Creating plots for {num_models} model(s)...")

# Check if we have any models to plot
if num_models == 0:
    print("\n⚠ WARNING: No T-SNE embeddings available to plot!")
    print("  Please run the T-SNE computation cells first.")
    print("\nSkipping visualization...")
else:
    # Check if EFA factor assignments are available
    factor_assignments_available = 'factor_assignments' in dir() and len(factor_assignments) > 0
    
    if not factor_assignments_available:
        print("\n⚠ WARNING: EFA factor assignments not available!")
        print("  Please run the PCA analysis cell first.")
        print("  Will show original factor coloring only.\n")
    
    # Create main combined plot with 2 rows (if PCA assignments available)
    if factor_assignments_available:
        # 2 rows: top = original factors, bottom = EFA factors
        fig_width = min(24, 8 * num_models)
        fig, axes = plt.subplots(2, min(3, num_models), figsize=(fig_width, 16))
        
        # Handle case of single model
        if min(3, num_models) == 1:
            axes = axes.reshape(2, 1)
    else:
        # Single row: original factors only
        fig_width = min(24, 8 * num_models)
        fig, axes = plt.subplots(1, min(3, num_models), figsize=(fig_width, 8))
        
        if min(3, num_models) == 1:
            axes = [axes]
    
    # Plot each model
    for idx, (model_size, embeddings_2d) in enumerate(sorted(all_tsne_embeddings.items())):
        if idx >= 3:  # Only plot first 3 models
            break
        
        # Get embedding dimension for titles
        embedding_dim = all_embeddings[model_size].shape[1]
        
        # ===== ROW 1: Original Factors =====
        if factor_assignments_available:
            ax = axes[0, idx]
        else:
            ax = axes[idx] if isinstance(axes, list) else axes
        
        # Plot each original factor with a different color
        for factor in unique_factors:
            indices = [i for i, f in enumerate(factors) if f == factor]
            ax.scatter(
                embeddings_2d[indices, 0],
                embeddings_2d[indices, 1],
                c=[factor_to_color[factor]],
                label=factor,
                alpha=0.6,
                s=80,
                edgecolors='white',
                linewidth=0.5
            )
        
        # Add labels
        for i in range(len(embeddings_2d)):
            ax.annotate(
                codes[i],
                (embeddings_2d[i, 0], embeddings_2d[i, 1]),
                fontsize=7,
                alpha=0.7,
                ha='center',
                va='bottom',
                xytext=(0, 3),
                textcoords='offset points'
            )
        
        ax.set_xlabel('T-SNE Component 1', fontsize=11)
        ax.set_ylabel('T-SNE Component 2', fontsize=11)
        ax.set_title(
            f'{model_size} Model - Original Factors\n({embedding_dim}D → 2D)',
            fontsize=13,
            fontweight='bold'
        )
        ax.grid(True, alpha=0.3, linestyle='--')
        
        # Add legend to rightmost plot
        if idx == min(2, num_models - 1):
            ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
        
        # ===== ROW 2: EFA Factors =====
        if factor_assignments_available:
            ax = axes[1, idx]
            
            # Get component assignments for this model
            factor_items = factor_assignments[model_size]
            n_factors = len(factor_items)
            
            # Create color map for components
            factor_colors_map = plt.cm.get_cmap('tab10', n_factors)
            factor_names = sorted(factor_items.keys())
            factor_to_color = {comp: factor_colors_map(i) for i, comp in enumerate(factor_names)}
            
            # Plot each component with a different color
            for factor_name in factor_names:
                # Get display name (LLM-generated or default)
                if 'factor_name_mappings' in dir() and model_size in factor_name_mappings:
                    display_name = factor_name_mappings[model_size].get(factor_name, factor_name)
                else:
                    display_name = factor_name

                indices = [item['index'] for item in factor_items[factor_name]]
                if len(indices) > 0:
                    ax.scatter(
                        embeddings_2d[indices, 0],
                        embeddings_2d[indices, 1],
                        c=[factor_to_color[factor_name]],
                        label=display_name,
                        alpha=0.6,
                        s=80,
                        edgecolors='white',
                        linewidth=0.5
                    )
            
            # Add labels
            for i in range(len(embeddings_2d)):
                ax.annotate(
                    codes[i],
                    (embeddings_2d[i, 0], embeddings_2d[i, 1]),
                    fontsize=7,
                    alpha=0.7,
                    ha='center',
                    va='bottom',
                    xytext=(0, 3),
                    textcoords='offset points'
                )
            
            ax.set_xlabel('T-SNE Component 1', fontsize=11)
            ax.set_ylabel('T-SNE Component 2', fontsize=11)
            ax.set_title(
                f'{model_size} Model - EFA Factors\n({embedding_dim}D → 2D)',
                fontsize=13,
                fontweight='bold'
            )
            ax.grid(True, alpha=0.3, linestyle='--')
            
            # Add legend to rightmost plot
            if idx == min(2, num_models - 1):
                ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
    
    # Overall title
    if factor_assignments_available:
        fig.suptitle(
            'T-SNE Visualization of Scale Item Embeddings',
            fontsize=16,
            fontweight='bold',
            y=0.995
        )
    else:
        fig.suptitle(
            'T-SNE Visualization of Scale Item Embeddings - Original Factors',
            fontsize=16,
            fontweight='bold',
            y=1.00
        )
    
    plt.tight_layout()
    
    # Save the combined figure
    model_names_str = "_".join(sorted(list(all_tsne_embeddings.keys())[:3]))
    if factor_assignments_available:
        filename = f"qwen3_tsne_combined_{model_names_str}_{timestamp}.png"
    else:
        filename = f"qwen3_tsne_original_{model_names_str}_{timestamp}.png"
    filepath = os.path.join(plots_dir, filename)
    plt.savefig(filepath, dpi=300, bbox_inches='tight')
    print(f"\n✓ Combined plot saved to: {filepath}")
    
    # Display the plot
    plt.show()
    
    # ===== Save Individual Plots =====
    if factor_assignments_available:
        print(f"\nCreating individual plots...")
        
        for model_size, embeddings_2d in sorted(all_tsne_embeddings.items()):
            embedding_dim = all_embeddings[model_size].shape[1]
            
            # Individual plot 1: Original Factors
            fig, ax = plt.subplots(1, 1, figsize=(10, 8))
            
            for factor in unique_factors:
                indices = [i for i, f in enumerate(factors) if f == factor]
                ax.scatter(
                    embeddings_2d[indices, 0],
                    embeddings_2d[indices, 1],
                    c=[factor_to_color[factor]],
                    label=factor,
                    alpha=0.6,
                    s=80,
                    edgecolors='white',
                    linewidth=0.5
                )
            
            for i in range(len(embeddings_2d)):
                ax.annotate(
                    codes[i],
                    (embeddings_2d[i, 0], embeddings_2d[i, 1]),
                    fontsize=7,
                    alpha=0.7,
                    ha='center',
                    va='bottom',
                    xytext=(0, 3),
                    textcoords='offset points'
                )
            
            ax.set_xlabel('T-SNE Component 1', fontsize=11)
            ax.set_ylabel('T-SNE Component 2', fontsize=11)
            ax.set_title(
                f'{model_size} Model - Original Factors\n({embedding_dim}D → 2D)',
                fontsize=14,
                fontweight='bold'
            )
            ax.grid(True, alpha=0.3, linestyle='--')
            ax.legend(loc='best', fontsize=10)
            
            plt.tight_layout()
            filename = f"qwen3_tsne_original_{model_size}_{timestamp}.png"
            filepath = os.path.join(plots_dir, filename)
            plt.savefig(filepath, dpi=300, bbox_inches='tight')
            plt.close()
            print(f"  ✓ {filepath}")
            
            # Individual plot 2: EFA Factors
            fig, ax = plt.subplots(1, 1, figsize=(10, 8))
            
            factor_items = factor_assignments[model_size]
            n_factors = len(factor_items)
            factor_colors_map = plt.cm.get_cmap('tab10', n_factors)
            factor_names = sorted(factor_items.keys())
            factor_to_color = {comp: factor_colors_map(i) for i, comp in enumerate(factor_names)}
            
            for factor_name in factor_names:
                # Get display name (LLM-generated or default)
                if 'factor_name_mappings' in dir() and model_size in factor_name_mappings:
                    display_name = factor_name_mappings[model_size].get(factor_name, factor_name)
                else:
                    display_name = factor_name

                indices = [item['index'] for item in factor_items[factor_name]]
                if len(indices) > 0:
                    ax.scatter(
                        embeddings_2d[indices, 0],
                        embeddings_2d[indices, 1],
                        c=[factor_to_color[factor_name]],
                        label=display_name,
                        alpha=0.6,
                        s=80,
                        edgecolors='white',
                        linewidth=0.5
                    )
            
            for i in range(len(embeddings_2d)):
                ax.annotate(
                    codes[i],
                    (embeddings_2d[i, 0], embeddings_2d[i, 1]),
                    fontsize=7,
                    alpha=0.7,
                    ha='center',
                    va='bottom',
                    xytext=(0, 3),
                    textcoords='offset points'
                )
            
            ax.set_xlabel('T-SNE Component 1', fontsize=11)
            ax.set_ylabel('T-SNE Component 2', fontsize=11)
            ax.set_title(
                f'{model_size} Model - EFA Factors\n({embedding_dim}D → 2D)',
                fontsize=14,
                fontweight='bold'
            )
            ax.grid(True, alpha=0.3, linestyle='--')
            ax.legend(loc='best', fontsize=10)
            
            plt.tight_layout()
            filename = f"qwen3_tsne_efa_{model_size}_{timestamp}.png"
            filepath = os.path.join(plots_dir, filename)
            plt.savefig(filepath, dpi=300, bbox_inches='tight')
            plt.close()
            print(f"  ✓ {filepath}")
    
    print("\n✓ Visualization complete!")